## 1. 初始化数据加载依赖

In [ ]:
!pip install boto3==1.26.53 -q
!pip install botocore==1.31.33 -q
!pip install mysql-connector-python -q

In [ ]:
import urllib.request
import tarfile
import os
import boto3
import requests
import pandas as pd
import mysql.connector

## 1. 配置数据库连接参数

In [ ]:
host = "nyu-mysql.cluster-cnfpwwtkftli.us-east-1.rds.amazonaws.com" # endpoint地址
port = 3306 # 端口
user = "awsuser" # 数据库连接用户吗
passwd = "4z(hFH-({u2Y*g:$BL{!D$Rp%H_!" # 数据库连接密码
database = "nyuog" #使用的数据库名称

## 2. 初始化数据库表结构
### 【注意】，只有首次初始化数据库表的时候才需要执行！！

In [ ]:
def init_data_table():
    try:
        mydb = mysql.connector.connect(host = host,
                                  port = port,
                                  user = user,
                                  passwd = passwd,
                                  database = database)
        sql_1 = """
            DROP TABLE IF EXISTS `edge`;
            """

        sql_2 = """
        CREATE TABLE `edge` (
              `id` bigint(20) unsigned zerofill NOT NULL AUTO_INCREMENT,
              `source` varchar(20) COLLATE utf8mb4_general_ci NOT NULL,
              `target` varchar(20) COLLATE utf8mb4_general_ci NOT NULL,
              `weight` float unsigned zerofill NOT NULL,
              `type` char(15) COLLATE utf8mb4_general_ci NOT NULL DEFAULT '',
              PRIMARY KEY (`id`)
            ) ENGINE=InnoDB AUTO_INCREMENT=20571 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
        """
        sql_3 = """
            SET FOREIGN_KEY_CHECKS = 1;
            """
        sql_4 = """
            DROP TABLE IF EXISTS `node`;
            """
        sql_5 = """
            CREATE TABLE `node` (
              `id` bigint(20) unsigned zerofill NOT NULL AUTO_INCREMENT COMMENT 'Node id',
              `name` varchar(40) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci NOT NULL COMMENT 'Node name',
              `type` char(15) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci NOT NULL DEFAULT '',
              `refer` varchar(20) COLLATE utf8mb4_general_ci DEFAULT NULL,
              `describe` varchar(255) COLLATE utf8mb4_general_ci DEFAULT NULL,
              PRIMARY KEY (`id`)
            ) ENGINE=InnoDB AUTO_INCREMENT=16103 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
            """
        sql_6 = """
            SET FOREIGN_KEY_CHECKS = 1;
            """
        mycursor = mydb.cursor()
        mycursor.execute(sql_1)
        mycursor.execute(sql_2)
        mycursor.execute(sql_3)
        mycursor.execute(sql_4)
        mycursor.execute(sql_5)
        mycursor.execute(sql_6)

        mydb.commit()
    except mysql.connector.Error as error:
        print("Failed to update record to database rollback: {}".format(error))
        # reverting changes because of exception
        mydb.rollback()
    finally:
        # closing database connection.
        if mydb.is_connected():
            mycursor.close()
            mydb.close()
            print("connection is closed")

init_data_table()


## 3.加载数据
去读文件夹：og_data/source 中的csv文件，并加载到数据库表的edge和node表中。
具体csv的格式请参考文件夹中的实力文件

### 3.1 定义加载方法

In [ ]:
# check if the node already exists.
def check_exist_node(nodelines, target) :
    # 遍历集合
    for row in nodelines:
        if row.split(',')[0] == target:
            return True

    return False


# create a row
def check_and_append_nodes(nodelines, target, label) :
    # print('check_and_append_nodes')
    if not check_exist_node(nodelines, target):
        # print('append')
        node = ['', '', '']
        node[0] = target
        node[1] = target
        node[2] = label
        node = ', '.join(node)
        nodelines.append(node)
        
        try_insert_node(target)
        

def try_insert_edge(source, target, weight):
    try:
        mydb = mysql.connector.connect(host = host,
                                  port = port,
                                  user = user,
                                  passwd = passwd,
                                  database = database
                                         )
        sql = "INSERT INTO edge (source, target, weight) VALUES (%s, %s, %s)"
        val = (source, target, weight)
        mycursor = mydb.cursor()
        mycursor.execute(sql, val)

        mydb.commit()

        # print(val)
        # print(mycursor.rowcount, "edge record inserted.")
    except mysql.connector.Error as error:
        print("Failed to update record to database rollback: {}".format(error))
        # reverting changes because of exception
        mydb.rollback()
    finally:
        # closing database connection.
        if mydb.is_connected():
            mycursor.close()
            mydb.close()
            # print("connection is closed")

def try_insert_node(nodename):
    try:
        mydb = mysql.connector.connect(host = host,
                                  port = port,
                                  user = user,
                                  passwd = passwd,
                                  database = database
                                         )
        sql = "INSERT INTO node (name, type) VALUES (%s, %s)"

        val = (nodename, '')
        mycursor = mydb.cursor()
        # print(val)
        mycursor.execute(sql, val)

        mydb.commit()

        # print(mycursor.rowcount, "node record inserted.")
    
    except mysql.connector.Error as error:
        print("Failed to update record to database rollback: {}".format(error))
        # reverting changes because of exception
        mydb.rollback()
    finally:
        # closing database connection.
        if mydb.is_connected():
            mycursor.close()
            mydb.close()
            # print("connection is closed")



def convert(sourcefile, targetnode, targetedge, label):
    print(sourcefile)
    nodelines = []
    # with open(sourcefile, 'r') as f:
    #     lines = f.readlines()
    
    lines = pd.read_csv(sourcefile)
    
    # print(len(lines))
    # print(lines)
    
    for index, row in lines.iterrows():
        # print(row)
        # edge_row = row.strip().split('\t')
        fromv = row[0]
        tov = row[1]
        weight = row[2]
        
        rowid =  fromv + tov

        try_insert_edge(fromv, tov, weight)
        
        # check the target node id is already exists. if not then append.
        check_and_append_nodes(nodelines, tov, label)
        # check the source node id is already exists. if not then append.
        check_and_append_nodes(nodelines, fromv, label)


#     with open(targetedge, 'w', newline='\r') as f:
#         # f.writelines(lines)
#         f.write('~id, ~from, ~to, weight:Double\r')
#         f.write('\r'.join(lines))
        
    # lines.to_csv(targetedge, index=False, mode='a', header=False)
    
    # df_nodes = pd.DataFrame(nodelines)
    # df_nodes.to_csv(targetnode, index=False, mode='a', header=False)


    # with open(targetnode, 'w', newline='\r') as f:
    #     # f.writelines(lines)
    #     f.write('~id, name:String, ~label\r')
    #     nodelines = '\r'.join(nodelines)
    #     for row in nodelines:
    #         f.write('\r'.join(row))



### 3.2 执行加载方法

In [ ]:

source_folder_path = 'og_data/source'
target_folder_path = 'og_data/target'

if not os.path.exists(target_folder_path):
    os.makedirs(target_folder_path)

# loop to handle the origional files.
for filename in os.listdir(source_folder_path):
    
    print(filename)
    # Convert the origional file to gremlin format file.
    convert(source_folder_path+"/"+filename,
            target_folder_path+"/node_"+filename,
            target_folder_path+"/edge_"+filename,
            filename)

## 4. 测试数据加载

In [ ]:
def try_select_node(nodename):
    try:
        mydb = mysql.connector.connect(host = host,
                                  port = port,
                                  user = user,
                                  passwd = passwd,
                                  database = database)
        mycursor = mydb.cursor()
        mycursor.execute("SELECT * FROM edge where source = 'E6SH66'")

        myresult = mycursor.fetchall()

        for x in myresult:
            print(x)
    except mysql.connector.Error as error:
        print("Failed to update record to database rollback: {}".format(error))
        # reverting changes because of exception
        mydb.rollback()
    finally:
        # closing database connection.
        if mydb.is_connected():
            mycursor.close()
            mydb.close()
            print("connection is closed")

try_select_node('E6SH66')